# T8. 利用 batch_step_fn 实现 R-DROP 和 adversarial training

&emsp;&emsp;相信你经过前面的教程学习，已经基本了解 `fastNLP` 各个组件的应用，本篇教程将通过 `R-DROP` 和 `adversarial training` 来实现自定义的 `batch_step_fn` 方法，来展示该方法的具体使用方法。

&emsp;&emsp;下面是默认的 `batch_step_fn` 函数的内容，包括调用 `train_step` 进行前向计算，反向传播，参数优化，梯度清零等过程。

In [ ]:
def batch_step_fn(trainer, batch):
    r"""
    针对一个 ``batch`` 的数据的训练过程；
    
    :param trainer: :class:`~fastNLP.core.controllers.Trainer` 实例；
    :param batch: 一个 ``batch`` 的数据；
    """
    outputs = trainer.train_step(batch)
    trainer.backward(outputs)
    trainer.step()
    trainer.zero_grad()

## R-DROP
&emsp;&emsp;本次实例将使用 `sst-2` 数据集来实现 `R-DROP` 的具体例子，关于数据处理部分因为前面系列的教程已经有明确说明，这里就不详细展开了。

In [ ]:
import torch
from datasets import load_dataset
from fastNLP import DataSet, cache_results, prepare_torch_dataloader, Trainer, Accuracy
from fastNLP.io import DataBundle
from torch import nn
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F

train_dataset, val_dataset, test_dataset = load_dataset("glue","sst2", split=["train", "validation", "test"])
train_dataset, val_dataset, test_dataset = train_dataset[:5000], val_dataset[:], test_dataset[:]

train_dataset = DataSet(train_dataset)
val_dataset = DataSet(val_dataset)
test_dataset = DataSet(test_dataset)

datasets = {"train":train_dataset,"val":val_dataset,"test":test_dataset}
data_bundle = DataBundle(datasets=datasets)
@cache_results('caches/cache.pkl')
def process_data(data_bundle, model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    def _process(review):
        encodings_review = tokenizer(review,padding="max_length",max_length=128,truncation=True)

        input_ids = encodings_review["input_ids"]
        attention_mask = encodings_review["attention_mask"]
        return {'input_ids': input_ids, 'attention_mask': attention_mask}

    data_bundle.apply_field_more(_process, field_name='sentence')

    return data_bundle, tokenizer

model_checkpoint = 'bert-base-uncased'
data_bundle, tokenizer = process_data(data_bundle, model_checkpoint, _refresh=True)


def collate_fn(batch):
    input_ids, atten_mask, labels = [], [], []
    max_length = [0] * 3

    for each_item in batch:
        input_ids.append(each_item["input_ids"])
        max_length[0] = max(max_length[0], len(each_item["input_ids"]))
        atten_mask.append(each_item["attention_mask"])
        max_length[1] = max(max_length[1], len(each_item["attention_mask"]))

        labels.append([each_item["label"]])
        max_length[2] = max(max_length[2], len([each_item["label"]]))

    for i in range(3):
        each = (input_ids, atten_mask, labels)[i]
        for item in each:
            item.extend([0] * (max_length[i] - len(item)))

    return {'input_ids': torch.cat([torch.tensor([item]) for item in input_ids], dim=0),
            'attention_mask': torch.cat([torch.tensor([item]) for item in atten_mask], dim=0),
            'labels': torch.cat([torch.tensor(item) for item in labels], dim=0)}
train_dataset = data_bundle.get_dataset('train')
evaluate_dataset = data_bundle.get_dataset('val')

train_dataloader = prepare_torch_dataloader(train_dataset, batch_size=16, shuffle=True,collate_fn=collate_fn)
evaluate_dataloader = prepare_torch_dataloader(evaluate_dataset, batch_size=16,collate_fn=collate_fn)

&emsp;&emsp;在这里我们会展示两种方法来实现 R-DROP，这里是第一种：

&emsp;&emsp;模型部分，所要修改的部分为在 `train_step` 函数中，要返回一个 `pred`，作为在 `batch_step_fn` 函数中计算最终 `loss` 时使用。

In [ ]:
class SeqClsModel(nn.Module):
    def __init__(self, num_labels, model_checkpoint):
        nn.Module.__init__(self)
        self.num_labels = num_labels
        self.back_bone = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,                                                                        num_labels=num_labels)

    def forward(self, input_ids, attention_mask, labels=None):

        output = self.back_bone(input_ids=input_ids,
                                attention_mask=attention_mask, labels=labels)

        return output

    def train_step(self, input_ids, attention_mask, labels):
        pred = self(input_ids, attention_mask, labels).logits
        loss =  self(input_ids, attention_mask, labels).loss
        return { "loss":loss,"pred":pred}

    def evaluate_step(self, input_ids, attention_mask, labels):
        pred = self(input_ids, attention_mask, labels).logits
        pred = torch.max(pred, dim=-1)[1]
        return {'pred': pred, 'target': labels}
model = SeqClsModel(num_labels=2, model_checkpoint=model_checkpoint)

&emsp;&emsp;在这里我们自定义一个 `batch_step_fn` 函数来实现 `R-DROP` 功能，首先，通过两次前向传播获得两个预测值，分别对其求交叉熵损失，并求平均值，获得 `ce_loss`，之后，对于两个 `pred` 求其KL散度损失，获得 `kl_loss`，将两者以一定的权重求和，并得到最终的 `loss`，在这里，我对于 `kl_loss` 的权重设置为 0.1。

In [ ]:
optimizers = AdamW(params=model.parameters(), lr=5e-5)

def batch_st_fn(trainer, batch):
    outputs = trainer.train_step(batch)
    pred = outputs["pred"]
    outputs = trainer.train_step(batch)
    pred2 = outputs["pred"]
    labels = batch["labels"]
    def compute_kl_loss(p, q):
        p_loss = F.kl_div(F.log_softmax(p, dim=-1), F.softmax(q, dim=-1), reduction='none')
        q_loss = F.kl_div(F.log_softmax(q, dim=-1), F.softmax(p, dim=-1), reduction='none')
        p_loss = p_loss.sum()
        q_loss = q_loss.sum()
        loss = (p_loss + q_loss) / 2
        return loss
    ce_loss = 0.5 * (F.cross_entropy(pred, labels) + F.cross_entropy(pred2, labels))
    kl_loss = compute_kl_loss(pred, pred2)
    loss = ce_loss + 0.1 * kl_loss
    outputs["loss"] = loss
    trainer.backward(outputs)
    trainer.step()
    trainer.zero_grad()
trainer = Trainer(
    model=model,
    driver='torch',
    device=5,  # 'cuda'
    n_epochs=5,
    optimizers=optimizers,
    train_dataloader=train_dataloader,
    evaluate_dataloaders=evaluate_dataloader,
    metrics={'acc': Accuracy()},
    batch_step_fn=batch_st_fn
)

trainer.run()

trainer.evaluator.run()

[22:44:24] INFO     Running evaluator sanity check for 2 batches.              ]8;id=279928;file:///remote-home/kychen/anaconda3/envs/pytorch2/lib/python3.8/site-packages/fastNLP/core/controllers/trainer.py\trainer.py]8;;\:]8;id=635596;file:///remote-home/kychen/anaconda3/envs/pytorch2/lib/python3.8/site-packages/fastNLP/core/controllers/trainer.py#661\661]8;;\

Output()

Output()

----------------------------- Eval. results on Epoch:1, Batch:0 -----------------------------

{
  "acc#acc": 0.91055
}

----------------------------- Eval. results on Epoch:2, Batch:0 -----------------------------

{
  "acc#acc": 0.880734
}

&emsp;&emsp;上述的方法在多卡情况下会有问题，因为分布式情况下，模型时不被允许多次前向计算或者多次反向传播的，所以上述的方法在多卡情况下无法运行，所以才有下面第二种方法来实现 `R-DROP`。

&emsp;&emsp;模型部分，所要修改的部分为在 `train_step` 函数中，要返回一个 `pred` 和一个 `pred2`，作为在 `batch_step_fn` 函数中计算最终 `loss` 时使用,在这个模型种，两次前向计算都是在模型内计算完成，所以不会出现上述的问题。


In [ ]:
class SeqClsModel(nn.Module):
    def __init__(self, num_labels, model_checkpoint):
        nn.Module.__init__(self)
        self.num_labels = num_labels
        self.back_bone = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,                                                                        num_labels=num_labels)

    def forward(self, input_ids, attention_mask, labels=None):

        output = self.back_bone(input_ids=input_ids,
                                attention_mask=attention_mask, labels=labels)

        return output

    def train_step(self, input_ids, attention_mask, labels):
        pred = self(input_ids, attention_mask, labels).logits
        pred2 = self(input_ids, attention_mask, labels).logits
        return {"loss":loss,"pred":pred,"pred2":pred2}

    def evaluate_step(self, input_ids, attention_mask, labels):
        pred = self(input_ids, attention_mask, labels).logits
        pred = torch.max(pred, dim=-1)[1]
        return {'pred': pred, 'target': labels}
model = SeqClsModel(num_labels=2, model_checkpoint=model_checkpoint)

&emsp;&emsp;在 `batch_step_fn` 中的变化不大，只是 `pred`,`pred2` 的来源是 `outputs` 的两个 `value` 值，其他不变。

In [3]:
optimizers = AdamW(params=model.parameters(), lr=5e-5)

def batch_st_fn(trainer, batch):
    outputs = trainer.train_step(batch)
    pred = outputs["pred"]
    pred2 = outputs["pred2"]
    labels = batch["labels"]
    def compute_kl_loss(p, q):
        p_loss = F.kl_div(F.log_softmax(p, dim=-1), F.softmax(q, dim=-1), reduction='none')
        q_loss = F.kl_div(F.log_softmax(q, dim=-1), F.softmax(p, dim=-1), reduction='none')
        p_loss = p_loss.sum()
        q_loss = q_loss.sum()
        loss = (p_loss + q_loss) / 2
        return loss
    ce_loss = 0.5 * (F.cross_entropy(pred, labels) + F.cross_entropy(pred2, labels))
    kl_loss = compute_kl_loss(pred, pred2)
    loss = ce_loss + 0.1 * kl_loss
    outputs["loss"] = loss
    trainer.backward(outputs)
    trainer.step()
    trainer.zero_grad()
trainer = Trainer(
    model=model,
    driver='torch',
    device=5,  # 'cuda'
    n_epochs=2,
    optimizers=optimizers,
    train_dataloader=train_dataloader,
    evaluate_dataloaders=evaluate_dataloader,
    metrics={'acc': Accuracy()},
    batch_step_fn=batch_st_fn
)

trainer.run()

trainer.evaluator.run()

[12:04:58] INFO     Running evaluator sanity check for 2 batches.              ]8;id=967677;file:///remote-home/kychen/anaconda3/envs/pytorch2/lib/python3.8/site-packages/fastNLP/core/controllers/trainer.py\trainer.py]8;;\:]8;id=878329;file:///remote-home/kychen/anaconda3/envs/pytorch2/lib/python3.8/site-packages/fastNLP/core/controllers/trainer.py#661\661]8;;\

Output()

Output()

----------------------------- Eval. results on Epoch:1, Batch:0 -----------------------------

{
  "acc#acc": 0.872706
}

----------------------------- Eval. results on Epoch:2, Batch:0 -----------------------------

{
  "acc#acc": 0.90367
}

Output()

{'acc#acc': 0.90367}

## adversarial training

&emsp;&emsp;下面，我们介绍一下在 `fastNLP` 中使用对抗训练的方法来自定义 `batch_step_fn`。

&emsp;&emsp;对抗训练，其基本的原理就是通过添加扰动构造一些对抗样本，放给模型去训练，以攻为守，提高模型在遇到对抗样本时的鲁棒性，同时一定程度也能提高模型的表现和泛化能力。
那么，什么样的样本才是好的对抗样本呢？对抗样本一般需要具有两个特点：

- 相对于原始输入，所添加的扰动是微小的；
- 能使模型犯错。

&emsp;&emsp;`GAN` 之父 Ian Goodfellow 在2015年的 `ICLR` 中  第一次提出了对抗训练这个概念，简而言之，就是在原始输入样本 x 上加一个扰动 $r_{adv}$，得到对抗样本后，用其进行训练。也就是说，问题可以被抽象成这么一个模型：

$$
min_\theta = - logP(y|x+r_{adv};\theta)
$$

&emsp;&emsp;其中 $r_{adv}$ 被定义为：

$$
    r_{adv} = \epsilon · sgn(\nabla_xL(\theta,x,y))
$$
					

&emsp;&emsp;Goodfellow还总结了对抗训练的两个作用：

- 提高模型应对恶意对抗样本时的鲁棒性；
- 作为一种 `regularization`，减少 `overfitting`，提高泛化能力。

##### Min-Max 公式

&emsp;&emsp;Madry 在2018年的 `ICLR` 中总结了之前的工作，并从优化的视角，将问题重新定义成了一个找鞍点的问题，也就是大名鼎鼎的 `Min-Max` 公式：

$$
min_\theta E_{(x,y)\sim D}[max_{r_{adv}\in S}L(\theta,x+r_{adv},y)]
$$


&emsp;&emsp;该公式分为两个部分，一个是内部损失函数的最大化，一个是外部经验风险的最小化。

- 内部 `max` 是为了找到 `worst-case` 扰动，也就是攻击，其中， `L` 为损失函数, `S` 为扰动的范围空间。
- 外部 `min` 是为了基于该攻击方式，找到最鲁棒的模型参数，也就是防御，其中 `D` 是输入样本的分布。

&emsp;&emsp;在CV任务，根据经验性的结论，对抗训练往往会使得模型在非对抗样本上的表现变差，然而神奇的是，在 NLP 任务中，模型的泛化能力反而变强了。因此**在 NLP 任务中，对抗训练的角色不再是为了防御基于梯度的恶意攻击，反而更多的是作为一种 regularization，提高模型的泛化能力**。

##### **Fast Gradient Method（FGM）**

$$
r_{adv} = \epsilon · g/||g||_2
$$

$$
g = \nabla_xL(\theta,x,y)
$$


- 该公式实际上就是取消了符号函数，用二范式做了一个scale

&emsp;&emsp;下面，我们具体来看 `FGM` 的代码实现：

In [3]:
class FGM():
    def __init__(self, model):
        self.model = model
        self.backup = {}

    def attack(self, epsilon=1., emb_name='word_embeddings'):
        # emb_name这个参数要换成你模型中embedding的参数名
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name:
                self.backup[name] = param.data.clone()
                norm = torch.norm(param.grad)
                if norm != 0 and not torch.isnan(norm):
                    r_at = epsilon * param.grad / norm
                    param.data.add_(r_at)

    def restore(self, emb_name='word_embeddings'):
        # emb_name这个参数要换成你模型中embedding的参数名
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name:
                assert name in self.backup
                param.data = self.backup[name]
        self.backup = {}
fgm = FGM(model)

&emsp;&emsp;在这里我们使用自定义的 `batch_step_fn` 来实现对抗训练。
- 首先通过一次前向计算和方向传播，得到 grad
- 在得到 grad 之后在 embedding 上添加对抗扰动
- 再重新进行一次前向计算反向传播，得到添加了对抗训练的梯度
- 恢复原本的 embedding 参数
- 进行参数更新和梯度清零

In [4]:
def batch_step_fn(trainer, batch):
    outputs = trainer.train_step(batch)

    trainer.backward(outputs)
    # 对抗训练
    fgm.attack()  
    outputs = trainer.train_step(batch)
    trainer.backward(outputs) 
    fgm.restore()  
    trainer.step()
    trainer.zero_grad()

In [5]:
optimizers = AdamW(params=model.parameters(), lr=5e-5)
trainer = Trainer(
    model=model,
    driver='torch',
    device=5,  # 'cuda'
    n_epochs=5,
    optimizers=optimizers,
    train_dataloader=train_dataloader,
    evaluate_dataloaders=evaluate_dataloader,
    metrics={'acc': Accuracy()},
    batch_step_fn=batch_step_fn,
)

trainer.run()

trainer.evaluator.run()

[14:16:26] INFO     Running evaluator sanity check for 2 batches.              ]8;id=558140;file:///remote-home/kychen/anaconda3/envs/pytorch2/lib/python3.8/site-packages/fastNLP/core/controllers/trainer.py\trainer.py]8;;\:]8;id=654168;file:///remote-home/kychen/anaconda3/envs/pytorch2/lib/python3.8/site-packages/fastNLP/core/controllers/trainer.py#661\661]8;;\

Output()

Output()

----------------------------- Eval. results on Epoch:1, Batch:0 -----------------------------

{
  "acc#acc": 0.862385
}

...

{
  "acc#acc": 0.876147
}

----------------------------- Eval. results on Epoch:5, Batch:0 -----------------------------

{
  "acc#acc": 0.866972
}

Output()

{'acc#acc': 0.866972}